# Script for predictions 

Here, we assume that there is a saved model which we want to use for predictions, and use it on a target image.

In [1]:
# Imports
import numpy as np, pandas as pd
import train as t
from src import helper_models as hm, helper_data as hd, helper_tts as htts
from importlib import reload
import keras

reload(t)
reload(hd)
reload(hm)
reload(htts)
# import os, keras, re, pathlib, numpy as np
# import matplotlib.pyplot as plt, seaborn as sns
# from src import helper_models, helper_data
# from PIL import Image

Using TensorFlow backend.



Running /opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py ['-f', '/home/marco/.local/share/jupyter/runtime/kernel-a184862a-3a9e-4a79-9f3a-c08c391a6fae.json']

Running /opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py ['-f', '/home/marco/.local/share/jupyter/runtime/kernel-a184862a-3a9e-4a79-9f3a-c08c391a6fae.json']


<module 'src.helper_tts' from '/mnt/volume2Tb/Dropbox/PROJECTS/MACHINE-LEARNING/AQUASCOPE/plankifier/src/helper_tts.py'>

In [2]:
datakind='image'


training_data=True
if datakind == 'image':
    
#     training_data='data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/'
    
    datapath='data/1_zooplankton_0p5x/validation/tommy_validation/images/'
    training_data=False
    
    modelpath ='./out/experiments/exps3-models/conv2-dropout-image/bestweights.hdf5'
    
    
elif datakind == 'mixed':
    
    datapath='data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/'
    modelpath  ='./out/experiments/exps3-models/mlp-mixed/bestweights.hdf5'
    



sim=t.Ctrain()
sim.params.training_data=training_data
sim.LoadModel(modelfile=modelpath)
# L and datakind need to come from LoadModel
L=128
class_select=None #['chaoborus','bosmina','unknown_plankton']

sim.LoadData(datapath=datapath, L=L, class_select=class_select, datakind=datakind, training_data=training_data)


Namespace(L=128, aug=False, bs=32, class_select=None, datakind='mixed', datapath='./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28//', dropout=None, earlyStopping=100, initial_epoch=0, layers=[256, 128], load=None, lr=5e-05, model='mlp', model_feat='mlp', model_image='mlp', opt='sgd', outpath='./out//', plot=False, saveModelName='keras_model.h5', testSplit=0.2, totEpochs=5, training_data=True, ttkind='mixed', verbose=False)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [ ]:
reload(htts)
sim.CreateTrainTestSets(ttkind=datakind, testSplit=1) # testsplit=1 means that all the data is in the test set


In [ ]:
sim.Report()


In [ ]:

# sim.tt.lb.classes_[predictions]

- Provare a fare la validation con i dati in Tommy Validation [fallito perche le classi sono diverse]
- Se si riesce a validare su Tommy validation, bisognerebbe rifare il training su tutto il dataset
- Bisognerebbe prendere i nomi delle classi dal modello caricato (cosi come il resto dei parametri) e non dai dati, in modo che il classification report funzioni anche se non carico tutto il dataset.

- Bisogna fare una funzione che prende una immagine e la predice, in modo da darlo agli ecologi

- Bisogna costruire un modello di ensemble, che secondo me con quello taglio la testa al toro



# Predicting the class of a single loaded image

In [ ]:


# Paths
modelpath='./out/conv2/2020-01-19_22h44m18s/'
modelpath='./out/conv2/2020-01-21_17h11m33s/'

argsname=modelpath+'args.txt'

# Test on Daphnia
target='daphnia'
im_names=['./data/zooplankton_trainingset_15oct/'+target+'/'+name for name in os.listdir('./data/zooplankton_trainingset_15oct/'+target+'/') if '.jpeg' in name]
print('We will predict the class of {} images belonging to the {} class'.format(len(im_names),target))

In [ ]:
# Read Arguments
with open(argsname,'r') as fargs:
    args=fargs.read()
print(args)
layers=[None,None]
for s in re.split('[\,,\),\(]',args):
    if 'height' in s:
        height=np.int64(re.search('(\d+)',s).group(1))
    if 'width' in s:
        width=np.int64(re.search('(\d+)',s).group(1))
    if 'depth' in s:
        depth=np.int64(re.search('(\d+)',s).group(1))
    if 'model' in s:
        modelname=re.search('=\'(.+)\'$',s).group(1)
    if 'resize' in s:
        resize=re.search('=\'(.+)\'$',s).group(1)
    if 'layers' in s: #first layer
        layers[0]=np.int64(re.search('=\[(.+)$',s).group(1))
    if re.match('^ \d+',s): #second layer
        layers[1]=np.int64(re.match('^ (\d+)',s).group(1))
    if 'datapath' in s:
        datapath=re.search('=\'(.+)\'$',s).group(1)
    if 'outpath' in s:
        outpath=re.search('=\'(.+)\'$',s).group(1)
    
classes = [ name for name in os.listdir(datapath) if os.path.isdir(os.path.join(datapath, name)) ]

# Double check that classes are correct
classes_dict=np.load(modelpath+'classes.npy',allow_pickle=True).item()


In [ ]:
from stat import S_ISREG, ST_CTIME, ST_MODE
# Choose model. We load the latest created .hdf5 file, since later is better
entries = [modelpath+'/'+entry for entry in os.listdir(modelpath) if '.hdf5' in entry]
entries = ((os.stat(path), path) for path in entries)
entries = ((stat[ST_CTIME], path) for stat, path in entries if S_ISREG(stat[ST_MODE]))
modelfile=sorted(entries)[-1][1]

# Initialize and load model
print('Model:',modelname)
model=keras.models.load_model(modelfile)

# model = keras.models.load_model(modelfile)
# opt = keras.optimizers.SGD(lr=0.001, nesterov=True)
# model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print('Loaded ',modelfile)

In [ ]:
def load_images(im_names, width, height, depth, modelname, resize):
    ''' 
    Function that loads a list of images given in im_names, and returns 
    them in a numpy format that can be used by the classifier
    '''
    npimages=np.ndarray((len(im_names),width,height,depth)) if modelname != 'mlp' else np.ndarray((len(im_names),width*height*depth))

    for i,im_name in enumerate(im_names):
        image = Image.open(im_name)
        if resize == 'acazzo':
            image = image.resize((width,height))
        else:
            # Set image's largest dimension to target size, and fill the rest with black pixels
            image,rescaled = helper_data.ResizeWithProportions(image, width) # width and height are assumed to be the same (assertion at the beginning)
            npimage=np.array(image.copy())
        if model == 'mlp':
            npimage = npimage.flatten()
        npimages[i]=npimage
        if len(im_names)==1:
            image.show()
        image.close()    
    return npimages/255.0


In [ ]:
# Load images
npimages=load_images(im_names, width, height, depth, modelname, resize)


In [ ]:
# Print prediction
predictions=model.predict(npimages).argmax(axis=1)
count=0
for i in range(len(npimages)):
    if classes_dict['name'][predictions[i]] == 'daphnia':
        count+=1
print('Accuracy: ', count/len(npimages))
